In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'Datasets\Movement\time_series_data_human_activities.csv')
df['magnitude'] = np.sqrt(df['x-axis']**2 + df['y-axis']**2 + df['z-axis']**2)

df['v_x'] = df['x-axis'].diff()
df['v_y'] = df['y-axis'].diff()
df['v_z'] = df['z-axis'].diff()

df['a_x'] = df['v_x'].diff()
df['a_y'] = df['v_y'].diff()
df['a_z'] = df['v_z'].diff()

df = pd.get_dummies(df, columns = ['activity'])
df = df.iloc[3:]
df = df.astype(int)


df['timestamp'] = df['timestamp'].apply(lambda x : int(x * 1e-8))

df.head()

,user,timestamp,x-axis,y-axis,z-axis,magnitude,v_x,v_y,v_z,a_x,a_y,a_z,activity_Downstairs,activity_Jogging,activity_Sitting,activity_Standing,activity_Upstairs,activity_Walking
3,1,13,-2,5,0,5,-3,0,0,2,1,0,0,0,0,0,0,1
4,1,13,-4,4,-1,6,-2,0,-1,0,0,-1,0,0,0,0,0,1
5,1,14,12,11,7,18,17,6,9,19,7,11,0,0,0,0,0,1
6,1,14,17,11,5,21,4,0,-1,-12,-6,-11,0,0,0,0,0,1
7,1,15,1,8,-5,10,-15,-3,-11,-20,-3,-9,0,0,0,0,0,1


In [3]:
df.columns

Index(['user', 'timestamp', 'x-axis', 'y-axis', 'z-axis', 'magnitude', 'v_x',
       'v_y', 'v_z', 'a_x', 'a_y', 'a_z', 'activity_Downstairs',
       'activity_Jogging', 'activity_Sitting', 'activity_Standing',
       'activity_Upstairs', 'activity_Walking'],
      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['x-axis', 'y-axis', 'z-axis', 'v_x', 'v_y', 'v_z', 'a_x', 'a_y', 'a_z']]
y = df[['activity_Downstairs',
       'activity_Jogging', 'activity_Sitting', 'activity_Standing',
       'activity_Upstairs', 'activity_Walking']][: X.shape[0] - 10]

def Real(new, window):
    X, y = [], []
    for i in range(new.shape[0] - window):
        row = np.array(new.iloc[i : i + window, :])
        X.append(row)
    return np.array(X)

X= Real(X, 10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True)

In [ ]:
# Input Features = [['x-axis', 'y-axis', 'z-axis', 'v_x', 'v_y', 'v_z', 'a_x', 'a_y', 'a_z']]
# Output onehot = [['activity_Downstairs', 'activity_Jogging', 'activity_Sitting', 'activity_Standing','activity_Upstairs', 'activity_Walking']]

# Input : [[ -2,   5,   0,  -3,   0,   0,   2,   1,   0],
    #    [ -4,   4,  -1,  -2,   0,  -1,   0,   0,  -1],
    #    [ 12,  11,   7,  17,   6,   9,  19,   7,  11],
    #    [ 17,  11,   5,   4,   0,  -1, -12,  -6, -11],
    #    [  1,   8,  -5, -15,  -3, -11, -20,  -3,  -9],
    #    [ -2,  15,   0,  -4,   7,   4,  11,  10,  16],
    #    [ -1,   9,  -1,   1,  -5,  -1,   5, -13,  -6],
    #    [  1,  10,   0,   2,   0,   1,   1,   6,   2],
    #    [  4,   9,  -1,   2,   0,   0,   0,  -1,  -1],
    #    [  4,   7,  -1,   0,  -2,   0,  -1,  -1,   0]]


# Output : [[0, 0, 0, 0, 0, 1]]

In [12]:
X_train.shape, y_train.shape

((805207, 10, 9), (805207, 6))

In [13]:
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint

model = keras.models.Sequential(
    [
        keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
        keras.layers.LSTM(units = 64, return_sequences = False),
        keras.layers.Dense(32, activation = 'relu'),
        keras.layers.Dense(6, activation = 'softmax')
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,222 (82.90 KB)

 Trainable params: 21,222 (82.90 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
cp = ModelCheckpoint('Model.keras', save_best_only = True, monitor = 'loss')

model.compile(loss = keras.losses.CategoricalCrossentropy(), optimizer = keras.optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])

model.fit(X_train, y_train, epochs = 20, callbacks = [cp])

Epoch 1/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 128s 5ms/step - accuracy: 0.8628 - loss: 0.3871
Epoch 2/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 125s 5ms/step - accuracy: 0.9306 - loss: 0.2033
Epoch 3/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 119s 5ms/step - accuracy: 0.9397 - loss: 0.1775
Epoch 4/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 125s 5ms/step - accuracy: 0.9433 - loss: 0.1649
Epoch 5/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 126s 5ms/step - accuracy: 0.9468 - loss: 0.1566
Epoch 6/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 124s 5ms/step - accuracy: 0.9488 - loss: 0.1500
Epoch 7/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 125s 5ms/step - accuracy: 0.9503 - loss: 0.1456
Epoch 8/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 125s 5ms/step - accuracy: 0.9507 - loss: 0.1433
Epoch 9/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 122s 5ms/step - accuracy: 0.9519 - loss: 0.1405
Epoch 10/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 124s 5ms/step - accuracy: 0.9525 - loss: 0.1379
Epoch 11/20
25163/25163 ━━━━━━━━━━━━━━━━━━━━ 124s 5ms/step - accuracy: 0.9530 -

In [18]:
model.evaluate(X_test, y_test)

8388/8388 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9493 - loss: 0.1513


[0.1496715247631073, 0.9499223232269287]